# Regression and Other Stories: Earnings

Predict respondents' yearly earnings using survey data from 1990. See Chapter 15 in Regression and Other Stories.

---

### Load packages

In [1]:
using DataFrames, StatsPlots, CSV, HTTP, StatsBase
using Distributions, Turing, MCMCChains
using StatsFuns: logistic

### Load data

In [2]:
data = "https://raw.githubusercontent.com/avehtari/ROS-Examples/master/Earnings/data/earnings.csv"
earnings  = CSV.File(HTTP.get(data).body, delim=",", missingstring="NA") |> DataFrame
first(earnings , 6)

,height,weight,male,earn,earnk,ethnicity,education,mother_education
,Int64,Int64?,Int64,Float64,Float64,String,Int64?,Int64?
1,74,210,1,50000.0,50.0,White,16,16
2,66,125,0,60000.0,60.0,White,16,16
3,64,126,0,30000.0,30.0,White,16,16
4,65,200,0,25000.0,25.0,White,17,17
5,63,110,0,50000.0,50.0,Other,16,16
6,68,165,0,62000.0,62.0,Black,18,18


## Compound discrete-continuous model

### Logistic regression on non-zero earnings

In [3]:
@model function m1(n, height, male, binary_earn)
    
    α ~ Normal(0, 10)
    βₕ ~ Normal(0, 5) 
    βₘ ~ Normal(0, 5)
    
    for i in 1:n
        v = logistic(α + βₕ * height[i] + βₘ * male[i])
        binary_earn[i] ~ Bernoulli(v)
    end
end;

In [4]:
binary_earn = Vector(earnings.earn .>0)
model_logistic = m1(nrow(earnings), earnings.height, earnings.male, binary_earn)
fit_2a = sample(model_logistic, NUTS(), 5000)
summarystats(fit_2a)

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/tburch/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/tburch/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/tburch/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference /Users/tburch/.julia/packages/Turing/uAz5c/src/inference/hmc.jl:195
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/tburch/.julia/packages/AdvancedHMC/MIx

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64 

           α   -2.6501    1.9725     0.0279    0.0540   1233.0273    0.9999
          βₕ    0.0681    0.0307     0.0004    0.0008   1216.9332    0.9998
          βₘ    1.7071    0.3262     0.0046    0.0086   1285.6211    1.0003


### Linear regression on log scale

In [5]:
@model function m2(height, male, log_earn)
    
    σ ~ Exponential(5)
    
    α ~ Normal(0, 10)
    βₕ ~ Normal(0, 5) 
    βₘ ~ Normal(0, 5)
    
    μ = α .+ βₕ * height .+ βₘ * male
    log_earn ~ MvNormal(μ, σ)
end;


In [6]:
valid = earnings.earn .!= 0.0
log_earn = log.(earnings.earn[valid])
model_log = m2(
    earnings.height[valid], 
    earnings.male[valid], 
    log_earn
)
fit_2b = sample(model_log, NUTS(), 5000)
summarystats(fit_2b)

┌ Info: Found initial step size
│   ϵ = 0.003125
└ @ Turing.Inference /Users/tburch/.julia/packages/Turing/uAz5c/src/inference/hmc.jl:195
Sampling: 100%|█████████████████████████████████████████| Time: 0:02:35


Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64 

           α    7.9498    0.5152     0.0073    0.0107   1655.7744    1.0001
          βₕ    0.0242    0.0080     0.0001    0.0002   1629.9706    1.0001
          βₘ    0.3690    0.0632     0.0009    0.0013   1795.0865    1.0000
           σ    0.8688    0.0153     0.0002    0.0003   2313.6295    0.9999


### Predictions for a new person

In [7]:
new = DataFrame(Dict(
        "height"=>[68],
        "male"=>[0],
        
        ))
m1_pred = m1(nrow(new), new.height, new.male, fill(missing, nrow(new)))
pred_2a=predict(m1_pred, fit_2a)

m2_pred = m2(new.height, new.male, missing)
pred_2b=predict(m2_pred, fit_2b)
pred = ifelse.(
    pred_2a["binary_earn[1]"].data .== 1, 
    exp.(pred_2b["log_earn[1]"].data),
    0
);